In [29]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import cn2an
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)

In [30]:
def check_for_nulls(df):
    """
    Test and report number of NAs in each column of the input data frame
    :param df: pandas.DataFrame
    :return: None
    """
    for col in df.columns.values:
        num_nans = np.sum(df[col].isnull())
        if num_nans > 0:
            print('%d Nans in col %s' % (num_nans, col))
    print('New shape of df: ', df.shape)

In [31]:
file_path = r"C:\Users\Dick\Desktop\Trading\001_Futures\Backtest\notebook\20200216_Cathay"
suffix = "_lvr_land_a.csv"

In [32]:
path_files = [f for f in listdir(file_path) if isfile(join(file_path, f))]

In [33]:
target_files = list()
for _f in path_files:
    if _f.endswith(suffix):
        target_files.append(_f)

In [34]:
print(target_files)

['a_lvr_land_a.csv', 'b_lvr_land_a.csv', 'e_lvr_land_a.csv', 'f_lvr_land_a.csv', 'h_lvr_land_a.csv']


In [35]:
# idx = pd.IndexSlice to_do
df_a = pd.read_csv(target_files[0], sep=",", header=0, skiprows=[1])
df_b = pd.read_csv(target_files[1], sep=",", header=0, skiprows=[1])
df_e = pd.read_csv(target_files[2], sep=",", header=0, skiprows=[1])
df_f = pd.read_csv(target_files[3], sep=",", header=0, skiprows=[1])
df_h = pd.read_csv(target_files[4], sep=",", header=0, skiprows=[1])

In [36]:
df_all = pd.concat([df_a, df_b, df_e, df_f, df_h])
df_all = df_all.reset_index(drop=True)

In [37]:
check_for_nulls(df_all)

628 Nans in col 都市土地使用分區
6122 Nans in col 非都市土地使用分區
6123 Nans in col 非都市土地使用編定
1189 Nans in col 移轉層次
1194 Nans in col 總樓層數
1361 Nans in col 主要用途
1189 Nans in col 主要建材
1300 Nans in col 建築完成年月
164 Nans in col 單價元平方公尺
3836 Nans in col 車位類別
4382 Nans in col 備註
New shape of df:  (6706, 28)


In [38]:
df_all.dtypes

鄉鎮市區             object
交易標的             object
土地區段位置建物區段門牌     object
土地移轉總面積平方公尺     float64
都市土地使用分區         object
非都市土地使用分區        object
非都市土地使用編定        object
交易年月日             int64
交易筆棟數            object
移轉層次             object
總樓層數             object
建物型態             object
主要用途             object
主要建材             object
建築完成年月          float64
建物移轉總面積平方公尺     float64
建物現況格局-房          int64
建物現況格局-廳          int64
建物現況格局-衛          int64
建物現況格局-隔間        object
有無管理組織           object
總價元               int64
單價元平方公尺         float64
車位類別             object
車位移轉總面積平方公尺     float64
車位總價元             int64
備註               object
編號               object
dtype: object

In [39]:
df_all.columns.values

array(['鄉鎮市區', '交易標的', '土地區段位置建物區段門牌', '土地移轉總面積平方公尺', '都市土地使用分區',
       '非都市土地使用分區', '非都市土地使用編定', '交易年月日', '交易筆棟數', '移轉層次', '總樓層數', '建物型態',
       '主要用途', '主要建材', '建築完成年月', '建物移轉總面積平方公尺', '建物現況格局-房', '建物現況格局-廳',
       '建物現況格局-衛', '建物現況格局-隔間', '有無管理組織', '總價元', '單價元平方公尺', '車位類別',
       '車位移轉總面積平方公尺', '車位總價元', '備註', '編號'], dtype=object)

In [41]:
# 將總樓層數轉化成數字
# df_a["總樓層數"] = df_a[df_a["總樓層數"].isnull() != True]["總樓層數"].apply(lambda x: cn2an.cn2an(x[:-1], "normal"))
df_all["總樓層數"] = df_all[df_all["總樓層數"].isnull() != True]["總樓層數"].apply(lambda x: cn2an.cn2an(x[:-1], "normal"))

In [42]:
# select 主要用途為"住家用"
_cond1 = df_all["主要用途"].str.contains("住家用", na=False)
# df_a.loc[df_a["主要用途"] == "住家用", "主要用途"]
# df_a.loc[df_a["主要用途"].str.contains("住家用", na=False), "主要用途"]
# df_all.loc[df_all["主要用途"].str.contains("住家用", na=False), "主要用途"]

# select 建物型態為 包含"住宅大樓"
_cond2 = df_all["主要用途"].str.contains("住家用", na=False)
# df_a.loc[df_a["建物型態"].str.contains("住宅大樓", na=False), "建物型態"]
# df_all.loc[df_all["建物型態"].str.contains("住宅大樓", na=False), "建物型態"]

# df_a.loc[df_a["總樓層數"].isnull() != True & df_a["總樓層數"] >= 13.0, "總樓層數"]
_cond3 = df_all["總樓層數"].isnull() != True
_cond4 = df_all["總樓層數"] >= 13.0

df_all_a = df_all.loc[_cond1 & _cond2 & _cond3 & _cond4  , :]

In [46]:
df_all_a = df_all_a.reset_index(drop=True)
df_all_a.to_csv("filter_a.csv", index=False)

In [47]:
# 分割交易筆棟數資訊成 3個個別欄位
transaction_pen_number = df_all["交易筆棟數"].str.split(r"\D+", expand=True)

df_all["交易筆棟數_土地"] = transaction_pen_number.iloc[:, 1]
df_all["交易筆棟數_建物"] = transaction_pen_number.iloc[:, 2]
df_all["交易筆棟數_車位"] = transaction_pen_number.iloc[:, 3]

# 轉換欄位類型
df_all = df_all.astype({'交易筆棟數_土地': 'int32',
                       '交易筆棟數_建物': 'int32',
                       '交易筆棟數_車位': 'int32'})

In [28]:
# 計算 總件數
print("總件數:{}".format(df_all.shape[0]))

# 計算 總車位數
print("總車位數:{}".format(df_all["交易筆棟數_車位"].sum()))

# 計算 平均總價元
print("平均總價元:{}".format(df_all["總價元"].sum() / df_all.shape[0]))

# 計算 平均車位總價元
print("平均車位總價元:{}".format(df_all["車位總價元"].sum() / df_all["交易筆棟數_車位"].sum()))

總件數:6706
總車位數:3360
平均總價元:15254421.616164628
平均車位總價元:926934.744047619


In [51]:
data_dict = {"總件數": [df_all.shape[0]],
             "總車位數": [df_all["交易筆棟數_車位"].sum()],
             "平均總價元": [df_all["總價元"].sum() / df_all.shape[0]],
             "平均車位總價元": [df_all["車位總價元"].sum() / df_all["交易筆棟數_車位"].sum()]}
filter_b = pd.DataFrame(data=data_dict)

In [52]:
filter_b.to_csv("filter_b.csv", index=False)